# Assignment 3 on Natural Language Processing

## Date : 30th Sept, 2020

### Instructor : Prof. Sudeshna Sarkar

### Teaching Assistants : Alapan Kuila, Aniruddha Roy, Anusha Potnuru, Uppada Vishnu

The central idea of this assignment is to use Naive Bayes classifier and LSTM based classifier and compare the models by accuracy on IMDB dataset.



Please submit with outputs. 

### **-18QE30008, Divyanshu Sheth**

In [32]:
# Gets the dataset
import os
import shutil
import zipfile
! rm -r sample_data
! wget "https://www.dropbox.com/s/a0na1oqn9ysb7fz/archive.zip?dl=1" 1> /dev/null
os.rename(os.path.join(os.getcwd(), 'archive.zip?dl=1'), os.path.join(os.getcwd(), 'archive.zip'))
with zipfile.ZipFile(os.path.join(os.getcwd(), 'archive.zip'), 'r') as zip_ref:
    zip_ref.extractall(os.getcwd())
! rm archive.zip

rm: cannot remove 'sample_data': No such file or directory
--2020-10-11 12:31:29--  https://www.dropbox.com/s/a0na1oqn9ysb7fz/archive.zip?dl=1
Resolving www.dropbox.com (www.dropbox.com)... 162.125.65.1, 2620:100:6021:1::a27d:4101
Connecting to www.dropbox.com (www.dropbox.com)|162.125.65.1|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/dl/a0na1oqn9ysb7fz/archive.zip [following]
--2020-10-11 12:31:29--  https://www.dropbox.com/s/dl/a0na1oqn9ysb7fz/archive.zip
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc694408d3851b5f09a23645d991.dl.dropboxusercontent.com/cd/0/get/BBDGnjDpg6_WL-PaBRzKmNLnCwPOLLLFqPI0iKts6PkMIMbq7EmFadrMp77Vcbj-cjuPCUhvI0Vx_GxZbNqZEgZ_7es1NA9-uz-4OCPuJt3d_Au01hEcFGijRVrMNPkCoSo/file?dl=1# [following]
--2020-10-11 12:31:30--  https://uc694408d3851b5f09a23645d991.dl.dropboxusercontent.com/cd/0/get/BBDGnjDpg6_WL-PaBRzKmNLnCwPOLLLFqPI0iKts6PkMIMbq7EmFa

In [47]:
# Imports
import re
import math
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
import keras
from sklearn.metrics import classification_report, accuracy_score
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize 
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import CountVectorizer
from collections import Counter, defaultdict
nltk.download('wordnet')
nltk.download('punkt')
nltk.download('stopwords')
stopw = stopwords.words('english')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [34]:
# Load the IMDB dataset
dataset = pd.read_csv('IMDB Dataset.csv')
print(dataset)

                                                  review sentiment
0      One of the other reviewers has mentioned that ...  positive
1      A wonderful little production. <br /><br />The...  positive
2      I thought this was a wonderful way to spend ti...  positive
3      Basically there's a family where a little boy ...  negative
4      Petter Mattei's "Love in the Time of Money" is...  positive
...                                                  ...       ...
49995  I thought this movie did a down right good job...  positive
49996  Bad plot, bad dialogue, bad acting, idiotic di...  negative
49997  I am a Catholic taught in parochial elementary...  negative
49998  I'm going to have to disagree with the previou...  negative
49999  No one expects the Star Trek movies to be high...  negative

[50000 rows x 2 columns]


# Preprocessing
PrePrecessing that needs to be done on lower cased corpus

1. Remove html tags
2. Remove URLS
3. Remove non alphanumeric character
4. Remove Stopwords
5. Perform stemming and lemmatization

You can use regex from re. 

In [35]:
def preprocess(raw_text):   
  cleantext = raw_text.lower()
  cleanr = re.compile('<.*?>')
  cleantext = re.sub(cleanr, ' ', cleantext)    
  cleantext = re.sub(r'http\S+', '', cleantext)  
  cleantext = re.sub(r'[^\w\s]', '', cleantext)  
  lemmatizer = WordNetLemmatizer()  
  cleanwords = [lemmatizer.lemmatize(word) for word in word_tokenize(cleantext) if word not in stopw]  
  cleantext = ' '.join(cleanwords)  
  return cleantext

In [36]:
def ppDataset(dataset):   # preprocess the entire dataset
  for i in range(len(dataset)):
    dataset.iloc[i][0] = preprocess(dataset.iloc[i][0])
  return dataset

In [37]:
dataset = ppDataset(dataset)
print("Preprocessed dataset:\n", dataset)

Preprocessed dataset:
                                                   review sentiment
0      one reviewer mentioned watching 1 oz episode y...  positive
1      wonderful little production filming technique ...  positive
2      thought wonderful way spend time hot summer we...  positive
3      basically there family little boy jake think t...  negative
4      petter matteis love time money visually stunni...  positive
...                                                  ...       ...
49995  thought movie right good job wasnt creative or...  positive
49996  bad plot bad dialogue bad acting idiotic direc...  negative
49997  catholic taught parochial elementary school nu...  negative
49998  im going disagree previous comment side maltin...  negative
49999  one expects star trek movie high art fan expec...  negative

[50000 rows x 2 columns]


In [38]:
# Print Statistics of Data like avg length of sentence, proposition of data w.r.t class labels
npos = 0
sumlen = 0
for i in range(len(dataset)):
  if dataset.iloc[i][1] == 'positive':
    npos += 1
  sumlen += len(word_tokenize(dataset.iloc[i][0]))

print("Average length of a review = {:.4f} words\n\nClass Distribution:\nPositive reviews: {} | Negative reviews: {} | Proportion of positive reviews: {:.4f}%".format(sumlen/len(dataset), npos, len(dataset) - npos, npos/len(dataset)*100))

Average length of a review = 120.2169 words

Class Distribution:
Positive reviews: 25000 | Negative reviews: 25000 | Proportion of positive reviews: 50.0000%


# Naive Bayes classifier

In [39]:
# get reviews column from df
reviews = dataset['review'].values

# get labels column from df
labels = dataset['sentiment'].values

In [44]:
# Use label encoder to encode labels. Convert to 0/1
encoder = LabelEncoder()
encoded_labels = encoder.fit_transform(labels)
dataset['encoded'] = encoded_labels
encoder_mapping = dict(zip(encoder.classes_, encoder.transform(encoder.classes_)))
labels = dataset['encoded']

In [45]:
# Split the data into train and test (80% - 20%). 
# Use stratify in train_test_split so that both train and test have similar ratio of positive and negative samples.
train_sentences, test_sentences, train_labels, test_labels = train_test_split(reviews, labels, test_size = 0.2, stratify = labels)
# train_sentences, test_sentences, train_labels, test_labels

Here there are two approaches possible for building vocabulary for the naive Bayes.
1. Take the whole data (train + test) to build the vocab. In this way while testing there is no word which will be out of vocabulary.
2. Take the train data to build vocab. In this case, some words from the test set may not be in vocab and hence one needs to perform smoothing so that one the probability term is not zero.
 
You are supposed to go by the 2nd approach.
 
Also building vocab by taking all words in the train set is memory intensive, hence you are required to build vocab by choosing the top 2000 - 3000 frequent words in the training corpus.

> $ P(x_i | w_j) = \frac{ N_{x_i,w_j}\, +\, \alpha }{ N_{w_j}\, +\, \alpha*d} $


$N_{x_i,w_j}$ : Number of times feature $x_i$ appears in samples of class $w_j$

$N_{w_j}$ : Total count of features in class $w_j$

$\alpha$ : Parameter for additive smoothing. Here consider $\alpha$ = 1

$d$ : Dimentionality of the feature vector  $x = [x_1,x_2,...,x_d]$. In our case its the vocab size.






In [48]:
# Use Count vectorizer to get frequency of the words
'''
max_features parameter : If not None, build a vocabulary that only consider the top max_features ordered by term frequency across the corpus.
vec = CountVectorizer(max_features = 3000)
X = vec.fit_transform(Sentence_list)
'''

vec = CountVectorizer(max_features = 3000)
X = vec.fit_transform(train_sentences)
counts = X.sum(axis = 0).A1
vocab = list(vec.get_feature_names())

freq = Counter(dict(zip(vocab, counts)))

print("The 100 most common words in the reviews are: ", freq.most_common(100), sep = '\n')

The 100 most common words in the reviews are: 
[('movie', 80720), ('film', 73550), ('one', 42981), ('like', 32039), ('time', 24037), ('good', 23217), ('character', 22282), ('story', 19782), ('even', 19653), ('get', 19466), ('would', 19061), ('make', 18957), ('see', 18933), ('really', 18342), ('scene', 16743), ('well', 15516), ('much', 15334), ('people', 14453), ('great', 14444), ('also', 14317), ('bad', 14299), ('first', 13773), ('show', 13771), ('dont', 13580), ('way', 13474), ('thing', 13111), ('made', 12325), ('think', 12085), ('could', 12062), ('life', 11849), ('go', 11558), ('know', 11403), ('watch', 11214), ('love', 10793), ('many', 10779), ('seen', 10619), ('plot', 10551), ('actor', 10545), ('two', 10402), ('say', 10364), ('never', 10288), ('end', 10226), ('look', 10128), ('acting', 10067), ('year', 9963), ('best', 9938), ('little', 9909), ('ever', 9451), ('better', 9050), ('man', 8945), ('take', 8909), ('come', 8821), ('still', 8682), ('work', 8534), ('find', 8124), ('part', 79

In [82]:
class Naive_Bayes:
    def __init__(self, classes):
      self.classes = classes

    def smoothing(self, word, tclass):          
      num = self.wcounts[tclass][word] + 1
      den = self.n_items[tclass] + len(self.vocab)
      return math.log(num / den)

    def fit(self, X, y):
        self.vocab = vocab
        self.wcounts = {}
        self.n_items = {}
        self.log_p = {}
        n = len(X)
        grouped = self.group(X, y)
        for c, data in grouped.items():
          self.n_items[c] = len(data)
          self.log_p[c] = math.log(self.n_items[c] / n) 
          self.wcounts[c] = defaultdict(lambda: 0)
          for txt in data:
            counts = Counter(nltk.word_tokenize(txt))
            for word, count in counts.items():
                self.wcounts[c][word] += count
        return self

    def predict(self, X):
        result = []
        for text in X:
          scores = {c: self.log_p[c] for c in self.classes}
          words = set(nltk.word_tokenize(text))
          for word in words:
              if word not in self.vocab: 
                continue
              for c in self.classes:
                log_wgc = self.smoothing(word, c)
                scores[c] += log_wgc
          result.append(max(scores, key = scores.get))
        return result

    def group(self, X, y):
      data = {}
      for c in self.classes:                          
        data[c] = X[np.where(y == c)]
      return data

In [83]:
nb = Naive_Bayes(classes = np.unique(labels)).fit(train_sentences, train_labels)

# Test the model on test set and report Accuracy
predicted_labels = nb.predict(test_sentences)
print("The accuracy of the Naive Bayes classifier is: \n{:.4f}%\n\n".format(accuracy_score(test_labels, predicted_labels) * 100))
print("\nThe classification report is as follows: \n\n", classification_report(test_labels, predicted_labels))

The accuracy of the Naive Bayes classifier is: 
84.8300%



The classification report is as follows: 

               precision    recall  f1-score   support

           0       0.84      0.85      0.85      5000
           1       0.85      0.84      0.85      5000

    accuracy                           0.85     10000
   macro avg       0.85      0.85      0.85     10000
weighted avg       0.85      0.85      0.85     10000



# *LSTM* based Classifier

Use the above train and test splits.

In [61]:
# Hyperparameters of the model
tokenizer = Tokenizer()
tokenizer.fit_on_texts(train_sentences)
word_index = tokenizer.word_index
vocab_size = len(tokenizer.word_index) + 1
oov_tok = '<OOK>'
embedding_dim = 100
max_length = 150
padding_type='post'
trunc_type='post'

In [62]:
# convert train dataset to sequence and pad sequences
train_sequences = tokenizer.texts_to_sequences(train_sentences)
train_padded = pad_sequences(train_sequences, padding='post', maxlen=max_length)

# convert Test dataset to sequence and pad sequences
test_sequences = tokenizer.texts_to_sequences(test_sentences)
test_padded = pad_sequences(test_sequences, padding='post', maxlen=max_length)

In [63]:
# model initialization
model = keras.Sequential([
    keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
    keras.layers.Bidirectional(keras.layers.LSTM(64)),
    keras.layers.Dense(24, activation='relu'),
    keras.layers.Dense(1, activation='sigmoid')
])

# compile model
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

# model summary
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 150, 100)          13621300  
_________________________________________________________________
bidirectional (Bidirectional (None, 128)               84480     
_________________________________________________________________
dense (Dense)                (None, 24)                3096      
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 25        
Total params: 13,708,901
Trainable params: 13,708,901
Non-trainable params: 0
_________________________________________________________________


In [64]:
#training the model
num_epochs = 5
history = model.fit(train_padded, train_labels, 
                    epochs=num_epochs, verbose=1, 
                    validation_split=0.1)

Epoch 1/5
1125/1125 [==============================] - 312s 278ms/step - loss: 0.3484 - accuracy: 0.8517 - val_loss: 0.2741 - val_accuracy: 0.8905
Epoch 2/5
1125/1125 [==============================] - 308s 274ms/step - loss: 0.1488 - accuracy: 0.9466 - val_loss: 0.3211 - val_accuracy: 0.8880
Epoch 3/5
1125/1125 [==============================] - 313s 278ms/step - loss: 0.0719 - accuracy: 0.9768 - val_loss: 0.4506 - val_accuracy: 0.8783
Epoch 4/5
1125/1125 [==============================] - 308s 273ms/step - loss: 0.0477 - accuracy: 0.9839 - val_loss: 0.4335 - val_accuracy: 0.8773
Epoch 5/5
1125/1125 [==============================] - 313s 278ms/step - loss: 0.0265 - accuracy: 0.9920 - val_loss: 0.5945 - val_accuracy: 0.8752


In [67]:
# Get probabilities
prediction = model.predict(test_padded)
print("Probabilities: ", prediction, sep='\n')

# Get labels based on probability 1 if p>= 0.5 else 0
for p in prediction:
  if p[0] >= 0.5:
    p[0] = 1
  else:
    p[0] = 0

prediction = prediction.astype('int32') 
print("\nLabels:", prediction, sep='\n')

# Calculate accuracy on Test data
print("\nAccuracy of the model: {:.4f}%\n".format(accuracy_score(test_labels, prediction) * 100))
print("Classification report: \n", classification_report(test_labels, prediction, labels = [0, 1]), sep='\n')


Probabilities: 
[[5.7497329e-01]
 [1.8772483e-04]
 [9.3421090e-01]
 ...
 [2.2846460e-04]
 [1.4536679e-03]
 [5.0371885e-04]]

Labels:
[[1]
 [0]
 [1]
 ...
 [0]
 [0]
 [0]]

Accuracy of the model: 87.4100%

Classification report: 

              precision    recall  f1-score   support

           0       0.89      0.85      0.87      5000
           1       0.86      0.90      0.88      5000

    accuracy                           0.87     10000
   macro avg       0.87      0.87      0.87     10000
weighted avg       0.87      0.87      0.87     10000



## Get predictions for random examples

In [69]:
# reviews on which we need to predict
sentence = ["The movie was very touching and heart whelming", 
            "I have never seen a terrible movie like this", 
            "the movie plot is terrible but it had good acting"]

# convert to a sequence
sequences = tokenizer.texts_to_sequences(sentence)

# pad the sequence
padded = pad_sequences(sequences, padding='post', maxlen=max_length)

# Get probabilities
prediction = model.predict(padded)
print("The probabilities are:", prediction, sep='\n')

# Get labels based on probability 1 if p>= 0.5 else 0
for p in prediction:
    if p[0] >=0.5:
        p[0] = 1
    else:
        p[0] = 0
prediction = prediction.astype('int32') 
print("\nLabels:", prediction, sep='\n')

The probabilities are:
[[0.94367266]
 [0.01868612]
 [0.00555304]]

Labels:
[[1]
 [0]
 [0]]


**Using the Naive Bayes model, we get an accuracy of ~85%, while an accuracy of ~87% is obtained on using the LSTM model. Therefore, we can conclude that LSTM is a better model as compared to Naive Bayes for this task.**